In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# 2. Create a temporary view of the DataFrame.
file_path = "Resources\home_sales_revised.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.createOrReplaceTempView("home_sales")
df.printSchema()

<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:2: SyntaxWarning: invalid escape sequence '\h'
C:\Users\David\AppData\Local\Temp\ipykernel_12064\485253360.py:2: SyntaxWarning: invalid escape sequence '\h'
  file_path = "Resources\home_sales_revised.csv"


root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
    SELECT 
        YEAR(date) AS year, 
        ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales 
    WHERE bedrooms = 4 
    GROUP BY YEAR(date)
    ORDER BY year
"""
result = spark.sql(query)
result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
    SELECT 
        date_built AS year, 
        ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales 
    WHERE bedrooms = 3 AND bathrooms = 3 
    GROUP BY date_built
    ORDER BY year
"""
result = spark.sql(query)
result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
    SELECT 
        date_built AS year, 
        ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales 
    WHERE bedrooms = 3 
      AND bathrooms = 3 
      AND floors = 2 
      AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY year
"""
result = spark.sql(query)
result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|298264.72|
|2015|297609.97|
|2016| 293965.1|
|2017|280317.58|
+----+---------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

query = """
    SELECT 
        view AS view_rating, 
        ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales 
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view_rating DESC
"""
result = spark.sql(query)
result.show()

end_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

--- 0.611644983291626 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
query = """
    SELECT 
        view AS view_rating, 
        ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales 
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view_rating DESC
"""

result_cached = spark.sql(query)
result_cached.show()
end_time = time.time()
cached_runtime = end_time - start_time
print(f"Runtime with cached data: {cached_runtime:.2f} seconds")

spark.catalog.uncacheTable("home_sales")
start_time = time.time()

result_uncached = spark.sql(query)
result_uncached.show()

end_time = time.time()
uncached_runtime = end_time - start_time

print("--- %s seconds ---" % (time.time() - start_time))


+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

Runtime with cached data: 0.93 seconds
+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         

In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
import os
output_path = "C:/Users/David/output/parquet"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
try:
    df.write.mode("overwrite").partitionBy("date_built").parquet(output_path)
    print(f"Data successfully written to {output_path}, partitioned by 'date_built'.")
except Exception as e:
    print("An error occurred while writing Parquet data:", e)

An error occurred while writing Parquet data: An error occurred while calling o55.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCo

In [ ]:
# 11. Read the formatted parquet data.
parquet_path = "C:/Users/David/output/parquet/*"
parquet_df = spark.read.parquet(parquet_path)
parquet_df.printSchema()
parquet_df.show(5)

In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

